# Geocoding com geopandas & geopy

Neste tutorial, vamos ver como realizar o *geocode* (obtenção dos pares de coordenadas a partir de endereços) para vários registros em uma tabela.
Vamos usar o [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html#) para carregar a tabela com os endereços, [Geopandas](https://geopandas.readthedocs.io/en/stable/) para acionar a biblioteca [geopy](https://geopy.readthedocs.io/en/stable/) que, por sua vez irá executar o geocode.
Como exemplo usarei uma tabela com três registros de hoteis na cidade do Rio de Janeiro e, a partir de seus respectivos endereços obteremos suas localizações por pares de coordenadas. **[Baixar planilha usada no exemplo](https://github.com/FelipeSBarros/Python-para-Geo-I/blob/master/ListaEnderecos.csv)**;

Aproveitando esse tema, vamos apresentar, na segunda parte deste tutorial como realizar o *geocode reverso* (obtenção do endereço a partir dos pares de coordenadas). Como não está implementado como ferramenta do geopandas, teremos que usar outros mecanismos, como a biblioteca [shapely](https://shapely.readthedocs.io/en/stable/) e, depois o **geopy**;

Para o geocode, usaremos como provedor a **API** [Nominatim](https://wiki.openstreetmap.org/wiki/Nominatim), do **Open Street Map**.

## 1. Geocoding

### Importando as bibliotecas

In [1]:
import geopandas as gpds
import pandas as pds # Para carregar a planilha com os endereços

In [2]:
gpds.tools.geocode("Rio de Janeiro, RJ", provider = "nominatim", user_agent="Intro Geocode")

,address,geometry
0,"Rio de Janeiro, Microrregião Rio de Janeiro, R...",POINT (-43.2093727 -22.9110137)


In [3]:
end = pds.read_csv("./ListaEnderecos.csv", encoding='UTF8')
end

,Hotel,Endereco
0,Copacabana Palace,"Avenida Atlântica, 1702, Copacabana - Rio de J..."
1,Hotel Marina,"Avenida Delfim Moreira 696, Leblon - Rio de Ja..."
2,Windsor,"Avenida Atlantica, 1020, Copacabana – Rio de J..."


In [4]:
gpds.tools.geocode(end["Endereco"], provider = "nominatim", user_agent="Intro Geocode", country_bias="Brazil")

,address,geometry
0,"Copacabana Palace, 1702, Avenida Atlântica, Co...",POINT (-43.1790751 -22.9670027)
1,"696, Avenida Delfim Moreira, Leblon, Zona Sul ...",POINT (-43.222646 -22.9869118)
2,"Avenida Atlântica, Copacabana, Zona Sul do Rio...",POINT (-43.189305 -22.984457)


In [5]:
end["Geom"] = gpds.tools.geocode(end["Endereco"], provider = "nominatim", user_agent="Intro Geocode", country_bias="Brazil")["geometry"]

In [6]:
end.head()

,Hotel,Endereco,Geom
0,Copacabana Palace,"Avenida Atlântica, 1702, Copacabana - Rio de J...",POINT (-43.1790751 -22.9670027)
1,Hotel Marina,"Avenida Delfim Moreira 696, Leblon - Rio de Ja...",POINT (-43.222646 -22.9869118)
2,Windsor,"Avenida Atlantica, 1020, Copacabana – Rio de J...",POINT (-43.189305 -22.984457)


In [7]:
end.Geom

0    POINT (-43.1790751 -22.9670027)
1     POINT (-43.222646 -22.9869118)
2      POINT (-43.189305 -22.984457)
Name: Geom, dtype: object

## Geocode reverso

In [8]:
import shapely.geometry
import geopy

In [9]:
end["Geom"][0].x

-43.1790751

In [10]:
end["Geom"][0].y

-22.9670027

In [11]:
geopy.geocoders.osm.Nominatim(user_agent="Intro Geocode").reverse(str(end["Geom"][0].y) + ", " + str(end["Geom"][0].x))

Location(Copacabana Palace, 1702, Avenida Atlântica, Copacabana, Zona Sul do Rio de Janeiro, Rio de Janeiro, Microrregião Rio de Janeiro, Região Metropolitana do Rio de Janeiro, RJ, Região Sudeste, 22021-001, Brasil, (-22.9670027, -43.1790751, 0.0))

In [12]:
end["y"] = end["Geom"].map(lambda p: p.y)
end["x"] = end["Geom"].map(lambda p: p.x)
end["yx"] = end["Geom"].map(lambda p: str(p.y) + ", " + str(p.x))

In [13]:
end.head()

,Hotel,Endereco,Geom,y,x,yx
0,Copacabana Palace,"Avenida Atlântica, 1702, Copacabana - Rio de J...",POINT (-43.1790751 -22.9670027),-22.967003,-43.179075,"-22.9670027, -43.1790751"
1,Hotel Marina,"Avenida Delfim Moreira 696, Leblon - Rio de Ja...",POINT (-43.222646 -22.9869118),-22.986912,-43.222646,"-22.9869118, -43.222646"
2,Windsor,"Avenida Atlantica, 1020, Copacabana – Rio de J...",POINT (-43.189305 -22.984457),-22.984457,-43.189305,"-22.984457, -43.189305"


In [14]:
geopy.geocoders.osm.Nominatim(user_agent="Intro Geocode").reverse(end[0:1]["yx"])

Location(Copacabana Palace, 1702, Avenida Atlântica, Copacabana, Zona Sul do Rio de Janeiro, Rio de Janeiro, Microrregião Rio de Janeiro, Região Metropolitana do Rio de Janeiro, RJ, Região Sudeste, 22021-001, Brasil, (-22.9670027, -43.1790751, 0.0))

In [15]:
end['RevAddress'] = end.apply(lambda row: geopy.geocoders.osm.Nominatim(user_agent="Intro Geocode").reverse(row['yx']),axis=1)

In [16]:
end.head()

,Hotel,Endereco,Geom,y,x,yx,RevAddress
0,Copacabana Palace,"Avenida Atlântica, 1702, Copacabana - Rio de J...",POINT (-43.1790751 -22.9670027),-22.967003,-43.179075,"-22.9670027, -43.1790751","(Copacabana Palace, 1702, Avenida Atlântica, C..."
1,Hotel Marina,"Avenida Delfim Moreira 696, Leblon - Rio de Ja...",POINT (-43.222646 -22.9869118),-22.986912,-43.222646,"-22.9869118, -43.222646","(696, Avenida Delfim Moreira, Leblon, Zona Sul..."
2,Windsor,"Avenida Atlantica, 1020, Copacabana – Rio de J...",POINT (-43.189305 -22.984457),-22.984457,-43.189305,"-22.984457, -43.189305","(Petrobras, Avenida Atlântica, Copacabana, Zon..."


## Geocoders disponíveis

In [17]:
geopy.geocoders.SERVICE_TO_GEOCODER

{'arcgis': geopy.geocoders.arcgis.ArcGIS,
 'azure': geopy.geocoders.azure.AzureMaps,
 'baidu': geopy.geocoders.baidu.Baidu,
 'bing': geopy.geocoders.bing.Bing,
 'databc': geopy.geocoders.databc.DataBC,
 'geocodeearth': geopy.geocoders.geocodeearth.GeocodeEarth,
 'geocodefarm': geopy.geocoders.geocodefarm.GeocodeFarm,
 'geonames': geopy.geocoders.geonames.GeoNames,
 'google': geopy.geocoders.googlev3.GoogleV3,
 'googlev3': geopy.geocoders.googlev3.GoogleV3,
 'here': geopy.geocoders.here.Here,
 'ignfrance': geopy.geocoders.ignfrance.IGNFrance,
 'liveaddress': geopy.geocoders.smartystreets.LiveAddress,
 'mapbox': geopy.geocoders.mapbox.MapBox,
 'nominatim': geopy.geocoders.osm.Nominatim,
 'opencage': geopy.geocoders.opencage.OpenCage,
 'openmapquest': geopy.geocoders.openmapquest.OpenMapQuest,
 'pelias': geopy.geocoders.pelias.Pelias,
 'photon': geopy.geocoders.photon.Photon,
 'pickpoint': geopy.geocoders.pickpoint.PickPoint,
 'tomtom': geopy.geocoders.tomtom.TomTom,
 'what3words': geopy.